<a href="https://colab.research.google.com/github/lucifernandes/acidentesPRF/blob/master/scrub_explore_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Dados abertos sobre Acidentes da Polícia Rodoviária Federal

Lendo as bases de dados

In [2]:
import pandas as pd

url = 'https://media.githubusercontent.com/media/lucifernandes/acidentesPRF/master/Bases'
bases = {'Ocorrencias': {'folder': 'Agrupados%20por%20ocorrencia', 
                         'files': ['datatran2007.csv', 'datatran2008.csv', 'datatran2009.csv', 'datatran2010.csv', 'datatran2011.csv', 'datatran2012.csv', 'datatran2013.csv', 'datatran2014.csv', 'datatran2015.csv', 'datatran2016_atual.csv', 'datatran2017.csv', 'datatran2018.csv', 'datatran2019.csv', 'datatran2020.csv']}, 
         'Pessoas': {'folder': 'Agrupados%20por%20pessoa', 
                     'files': ['acidentes2007.csv', 'acidentes2008.csv', 'acidentes2009.csv', 'acidentes2010.csv', 'acidentes2011.csv', 'acidentes2012.csv', 'acidentes2013.csv', 'acidentes2014.csv', 'acidentes2015.csv', 'acidentes2016_atual.csv', 'acidentes2017.csv', 'acidentes2018.csv', 'acidentes2019.csv', 'acidentes2020.csv']}, 
         'Acidentes': {'folder': 'Agrupados%20por%20pessoa%20-%20Todas%20as%20causas%20e%20tipos%20de%20acidentes%20(desde%202017)', 
                       'files': ['acidentes2017_todas_causas_tipos.csv', 'acidentes2018_todas_causas_tipos.csv', 'acidentes2019_todas_causas_tipos.csv', 'acidentes2020_todas_causas_tipos.csv']}
         }

In [ ]:
# Mostra o link de todas as bases
for base in bases.keys():
  for arq in bases[base]['files']:
    print(f'{url}/{bases[base]["folder"]}/{arq}?raw=true')

# **Etapa Limpeza**

Para a etapa de limpeza, as bases de dados foram estudadas separadamente, como será mostrado a seguir:

## Base 1: **Agrupados por ocorrência**

In [ ]:
# Observações para a reunião (Hugo)

# Tranformação dos atributos categóricos para numéricos (One-hot ou colocar números)
# Escolher atributos que podem gerar enviesamento(Naturalidade, Sexo) 
# Escolher atributos que serão estimados (Classificação e regressão)
# Normalização/padronização dos atributos numéricos
# Escolher estratégias para solução de dados faltantes (Ignorada, Não informada, (null), NULL, NaN)
# Analisar possível problema de Id's repetidos

# Base de Pessoas
# pesid repetido
#   Naturalidade = "Não Informado"
#   Marca = (null)

# Base de Ocorrencias 
# Id repetido
#   classificacao_acidente = (null)
#   condicao_metereologica = "Ignorada"

## Base 2: **Agrupados por pessoa**

# Limpeza na base de pessoas

* Atributos para **MANTER**: **id**, **pesid**, **data_inversa**, **dia_semana**, **horario**, **uf**, **br**, **km**, **municipio**, **causa_acidente**, **tipo_acidente**, **classificação_acidente**, **condicao_metereologica**, **sentido_via**, **tracado_via**, **tipo_veiculo**, **tipo_envolvido**, **estado_fisico**, **idade**, **sexo**, **ilesos**, **feridos_leves**, **feridos_graves**, **mortos**.

* Atributos para **REMOVER**: **fase_dia**, **sentido_via**, **uso_solo**, **id_veiculo**, **marca**, **ano_fabricacao_veiculo**, **latitude**, **longitude**, **regional**, **delegacia**, **uop**.




In [97]:
# IMPORTAÇÃO DAS 4 ÚLTIMAS BASES DE PESSOAS
dfs = list()
t = len(bases['Pessoas']['files'])
for i in range(t-4, t):
  dfs.append(pd.read_csv(f'{url}/{bases["Pessoas"]["folder"]}/{bases["Pessoas"]["files"][i]}?raw=true', encoding='ISO-8859-1', sep=';'))

dfPessoas = pd.concat(dfs)
dfPessoas.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 609882 entries, 0 to 78411
Data columns (total 35 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      609882 non-null  float64
 1   pesid                   609878 non-null  float64
 2   data_inversa            609882 non-null  object 
 3   dia_semana              609882 non-null  object 
 4   horario                 609882 non-null  object 
 5   uf                      609882 non-null  object 
 6   br                      608891 non-null  float64
 7   km                      608891 non-null  object 
 8   municipio               609882 non-null  object 
 9   causa_acidente          609882 non-null  object 
 10  tipo_acidente           609882 non-null  object 
 11  classificacao_acidente  609882 non-null  object 
 12  fase_dia                609882 non-null  object 
 13  sentido_via             609882 non-null  object 
 14  condicao_metereologic

In [98]:
# REMOÇÃO DE ATRIBUTOS
dfPessoas = dfPessoas.drop(columns=['fase_dia', 'sentido_via', 'uso_solo', 'id_veiculo', 'marca', 'ano_fabricacao_veiculo', 'latitude', 'longitude', 'regional', 'delegacia', 'uop'])
dfPessoas.shape

(609882, 24)

In [61]:
# VERIFICAÇÃO DE NULOS
def testa_valores_nulos(dataframe, atributo):
  if sum(dataframe[atributo].isnull()) == 0:
    print('\033[94m', atributo, ' => OK (não tem valores nulos)')
  else:
    print('\033[93m', atributo, ' => *** TEM VALORES NULOS')

def testa_valores_null(dataframe, atributo):
  if sum(dataframe[atributo] == '(null)') == 0:
    print('\033[92m', atributo, ' => OK (não tem valores "(null)")')
  else:
    print('\033[91m', atributo, ' => *** TEM VALORES "(null)"')

print('-'*30, 'VALORES NULOS', '-'*30)
for attr in dfPessoas.columns:
  testa_valores_nulos(dfPessoas, attr)

print('\033[0m', '-'*30, 'VALORES (NULL)', '-'*30)
for attr in dfPessoas.columns:
  testa_valores_null(dfPessoas, attr)

------------------------------ VALORES NULOS ------------------------------
 id  => OK (não tem valores nulos)
 pesid  => *** TEM VALORES NULOS
 data_inversa  => OK (não tem valores nulos)
 dia_semana  => OK (não tem valores nulos)
 horario  => OK (não tem valores nulos)
 uf  => OK (não tem valores nulos)
 br  => *** TEM VALORES NULOS
 km  => *** TEM VALORES NULOS
 municipio  => OK (não tem valores nulos)
 causa_acidente  => OK (não tem valores nulos)
 tipo_acidente  => OK (não tem valores nulos)
 classificacao_acidente  => OK (não tem valores nulos)
 condicao_metereologica  => OK (não tem valores nulos)
 tipo_pista  => OK (não tem valores nulos)
 tracado_via  => OK (não tem valores nulos)
 tipo_veiculo  => OK (não tem valores nulos)
 tipo_envolvido  => OK (não tem valores nulos)
 estado_fisico  => OK (não tem valores nulos)
 idade  => *** TEM VALORES NULOS
 sexo  => OK (não tem valores nulos)
 ilesos  => OK (não tem valores nulos)
 feridos_leves  => OK (não tem valores nulos)
 feridos

/usr/local/lib/python3.6/dist-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


 id  => OK (não tem valores "(null)")
 pesid  => OK (não tem valores "(null)")
 data_inversa  => OK (não tem valores "(null)")
 dia_semana  => OK (não tem valores "(null)")
 horario  => OK (não tem valores "(null)")
 uf  => OK (não tem valores "(null)")
 br  => OK (não tem valores "(null)")
 km  => OK (não tem valores "(null)")
 municipio  => OK (não tem valores "(null)")
 causa_acidente  => OK (não tem valores "(null)")
 tipo_acidente  => OK (não tem valores "(null)")
 classificacao_acidente  => OK (não tem valores "(null)")
 condicao_metereologica  => OK (não tem valores "(null)")
 tipo_pista  => OK (não tem valores "(null)")
 tracado_via  => OK (não tem valores "(null)")
 tipo_veiculo  => OK (não tem valores "(null)")
 tipo_envolvido  => OK (não tem valores "(null)")
 estado_fisico  => OK (não tem valores "(null)")
 idade  => OK (não tem valores "(null)")
 sexo  => OK (não tem valores "(null)")
 ilesos  => OK (não tem valores "(null)")
 feridos_leves  => OK (não tem valores "(null)"

In [100]:
# VERIFICAÇÃO DE TIPOS
def testa_tipos(dataframe, atributo, tipo):
  s = sum(dataframe[atributo].map(type) != tipo)
  if s == 0: 
    print('\033[92m', f'{atributo} => todos são {tipo}')
  else: 
    print('\033[91m', f'{atributo} => *** {s} ATRIBUTOS NÃO SÃO {tipo}')

atributos_str = ['data_inversa', 'dia_semana', 'horario', 'uf', 'municipio', 'causa_acidente', 'tipo_acidente', 'classificacao_acidente', 'condicao_metereologica', 'tipo_pista', 'tracado_via', 'tipo_veiculo', 'tipo_envolvido', 'estado_fisico', 'sexo']
atributos_int = ['id', 'pesid', 'br', 'idade', 'ilesos', 'feridos_leves', 'feridos_graves', 'mortos']
atributos_float = ['km']

print('\033[0m', 'Atributos que devem ser STR')
for attr in atributos_str:
    testa_tipos(dfPessoas, attr, str)

print('\033[0m', 'Atributos que devem ser INT')
for attr in atributos_int:
    testa_tipos(dfPessoas, attr, int)

print('\033[0m', 'Atributos que devem ser FLOAT')
for attr in atributos_float:
    testa_tipos(dfPessoas, attr, float)

print('\033[0m', 'Quantidade de DATA INVERSA FORA DO PADRÃO AAAA-MM-DD =', sum(~dfPessoas['data_inversa'].str.match('[0-9]{4}-[0-9]{2}-[0-9]{2}')))
print('\033[0m', 'Quantidade de HORÁRIO FORA DO PADRÃO HH:MM:SS =', sum(~dfPessoas['horario'].str.match('[0-9]{2}:[0-9]{2}:[0-9]{2}')))

 Atributos que devem ser STR
 data_inversa => todos são <class 'str'>
 dia_semana => todos são <class 'str'>
 horario => todos são <class 'str'>
 uf => todos são <class 'str'>
 municipio => todos são <class 'str'>
 causa_acidente => todos são <class 'str'>
 tipo_acidente => todos são <class 'str'>
 classificacao_acidente => todos são <class 'str'>
 condicao_metereologica => todos são <class 'str'>
 tipo_pista => todos são <class 'str'>
 tracado_via => todos são <class 'str'>
 tipo_veiculo => todos são <class 'str'>
 tipo_envolvido => todos são <class 'str'>
 estado_fisico => todos são <class 'str'>
 sexo => todos são <class 'str'>
 Atributos que devem ser INT
 id => *** 609882 ATRIBUTOS NÃO SÃO <class 'int'>
 pesid => *** 609882 ATRIBUTOS NÃO SÃO <class 'int'>
 br => *** 609882 ATRIBUTOS NÃO SÃO <class 'int'>
 idade => *** 609882 ATRIBUTOS NÃO SÃO <class 'int'>
 ilesos => todos são <class 'int'>
 feridos_leves => todos são <class 'int'>
 feridos_graves => todos são <class 'int'>
 morto

In [101]:
# LIMPEZA
## Remove regitros com atributos br e km NaN
t = len(dfPessoas.id)
dfPessoas = dfPessoas[dfPessoas['br'].notna()]
dfPessoas = dfPessoas[dfPessoas['km'].notna()] # Nem precisava, onde br é NaN, km também é
print(f'Removidos {t - len(dfPessoas.id)} registros.')

## Conversões
dfPessoas['id'] = dfPessoas['id'].astype(int)

qtd_pesid_nan = len(dfPessoas[dfPessoas['pesid'].isna()]['pesid'])
dfPessoas.loc[dfPessoas['pesid'].isna(), 'pesid'] = [dfPessoas['pesid'].max() + 1 + i for i in range(qtd_pesid_nan)] # Substituindo valores NaN por PESIDs novos
dfPessoas['pesid'] = dfPessoas['pesid'].astype(int)

media_idade = dfPessoas.loc[dfPessoas['idade'] <= 110, 'idade'].mean()
dfPessoas.loc[(dfPessoas['idade'].isna()) | (dfPessoas['idade'] > 110), 'idade'] = media_idade
dfPessoas['idade'] = dfPessoas['idade'].astype(int)

Removidos 991 registros.


In [102]:
dfPessoas.head()

,id,pesid,data_inversa,dia_semana,horario,uf,br,km,municipio,causa_acidente,tipo_acidente,classificacao_acidente,condicao_metereologica,tipo_pista,tracado_via,tipo_veiculo,tipo_envolvido,estado_fisico,idade,sexo,ilesos,feridos_leves,feridos_graves,mortos
0,8,1,2017-01-01,domingo,00:00:00,PR,376.0,112,PARANAVAI,Fenômenos da Natureza,Queda de ocupante de veículo,Com Vítimas Feridas,Chuva,Simples,Reta,Motocicleta,Condutor,Lesões Graves,19,Masculino,0,0,1,0
1,9,955,2017-01-01,domingo,00:01:00,SC,101.0,234,PALHOCA,Falta de Atenção à Condução,Colisão com objeto estático,Sem Vítimas,Chuva,Dupla,Curva,Automóvel,Condutor,Ileso,35,Masculino,1,0,0,0
2,11,3,2017-01-01,domingo,00:00:00,PR,153.0,"56,9",SANTO ANTONIO DA PLATINA,Animais na Pista,Capotamento,Com Vítimas Feridas,Garoa/Chuvisco,Simples,Reta,Automóvel,Passageiro,Lesões Leves,27,Masculino,0,1,0,0
3,11,2,2017-01-01,domingo,00:00:00,PR,153.0,"56,9",SANTO ANTONIO DA PLATINA,Animais na Pista,Capotamento,Com Vítimas Feridas,Garoa/Chuvisco,Simples,Reta,Automóvel,Condutor,Lesões Leves,27,Feminino,0,1,0,0
4,12,1499,2017-01-01,domingo,00:00:00,GO,153.0,435,ANAPOLIS,Avarias e/ou desgaste excessivo no pneu,Tombamento,Com Vítimas Feridas,Céu Claro,Dupla,Reta,Motocicleta,Condutor,Lesões Graves,24,Masculino,0,0,1,0


# **Etapa Exploração**